# Feature Importance
Pracuj se stejnými daty jako na cvičení, tj. se souborem soybean-1-rot.csv.
Rozhodovací strom nám umožňuje nahlédnout do pravidel, podle kterých postupuje ve klasifikaci. Díky tomu se často pokládá za velice průhledný nebo dobře interpretovatelný algoritmus.

- Podívej se na atribut feature_importances_ (clf.feature_importances_), který říká, které vstupní proměnné model použil pro rozhodování. Některé budou mít nulovou hodnotu, to znamená, že vůbec potřeba nejsou. Atribut nám dá jen seznam čísel seřazený podle vstupních proměnných, ale ne jejich jména. Ty získáš například z OneHotEncoder (atribut feature_names_in_, takže například níže by se jednalo o oh_encoder.feature_names_in_)
- Která vstupní proměnná má největší "důležitost"?
- Stačí nám tato proměnná pro úspěšnou klasifikaci? Jaký je rozdíl mezi hodnotou f1_score při použití všech proměnných a jen této jedné "nejdůležitější" proměnné?

In [2]:
import pandas

from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import requests

r = requests.get(
    "https://raw.githubusercontent.com/lutydlitatova/czechitas-datasets/main/datasets/soybean-2-rot.csv"
)
open("soybean-2-rot.csv", "wb").write(r.content)

data = pandas.read_csv("soybean-2-rot.csv")

data.head()

,plant-stand,temp,hail,crop-hist,severity,seed-tmt,germination,plant-growth,leaves,leafspots-halo,...,fruiting-bodies,external-decay,mycelium,seed,mold-growth,seed-discolor,seed-size,shriveling,roots,class
0,normal,gt-norm,yes,same-lst-yr,pot-severe,fungicide,90-100,abnorm,abnorm,absent,...,absent,absent,absent,norm,absent,absent,norm,absent,norm,charcoal-rot
1,normal,norm,no,same-lst-yr,pot-severe,fungicide,80-89,abnorm,abnorm,absent,...,absent,absent,absent,norm,absent,absent,norm,absent,norm,charcoal-rot
2,normal,norm,yes,same-lst-yr,pot-severe,none,90-100,abnorm,abnorm,absent,...,absent,absent,absent,norm,absent,absent,norm,absent,norm,charcoal-rot
3,normal,norm,no,same-lst-sev-yrs,pot-severe,fungicide,90-100,abnorm,abnorm,absent,...,absent,absent,absent,norm,absent,absent,norm,absent,norm,charcoal-rot
4,normal,gt-norm,yes,same-lst-yr,pot-severe,fungicide,80-89,abnorm,abnorm,absent,...,absent,absent,absent,norm,absent,absent,norm,absent,norm,charcoal-rot


In [3]:
X = data.drop(columns=["class"])
y = data["class"]

oh_encoder = OneHotEncoder()
X = oh_encoder.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=0
)

clf = DecisionTreeClassifier(max_depth=3, min_samples_leaf=1)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(f1_score(y_test, y_pred, average="weighted"))

0.9279569892473118


Vykresli graf, ze kterého je vidět rozložení hodnot této jedné nejdůležitější proměnné. Můžeš využít groupby nebo pivot_table v kombinaci s metodou plot, nebo například sns.countplot (kde sns je modul seaborn). Příklad takového grafu s jinou proměnnou najdeš zde.

# Dobrovolný doplněk
Vykresli graf, ze kterého je vidět rozložení hodnot této jedné nejdůležitější proměnné. Můžeš využít groupby nebo pivot_table v kombinaci s metodou plot, nebo například sns.countplot (kde sns je modul seaborn).